# L3 M8.2: Real-Time Financial Data Enrichment

**From:** FinanceAI M8.2 - Real-Time Financial Data Enrichment

**Script:** [Augmented_Finance_AI_M8_2_RealTime_Financial_Data_Enrichment (1).md](https://github.com/yesvisare/financial_ai_ccc_l2/blob/main/Augmented_Finance_AI_M8_2_RealTime_Financial_Data_Enrichment%20(1).md)

## Learning Arc

### The Problem

Imagine you're an investment analyst at 9:45 AM on earnings day. You query your RAG system:

*"What's Apple's current stock price and how does it compare to analyst expectations?"*

Your system retrieves a document from yesterday that says Apple is trading at $185. But the market opened 15 minutes ago, and Apple is actually at $192—up 3.8% on better-than-expected earnings announced this morning.

**Your RAG system just gave you stale data.** In financial services, 15-minute-old information can cost millions in missed opportunities or lead to trading violations.

### What You'll Build

A **real-time financial data enrichment layer** that:
- Integrates live market data from yfinance (free) with 15-minute delay
- Implements intelligent caching with Redis (60% cost savings)
- Handles market hours correctly (no stale after-hours data)
- Degrades gracefully when APIs fail

### Key Concepts

1. **Financial Data Enrichment** - Augmenting historical documents with current market data
2. **Data Freshness Hierarchy** - Understanding when real-time data matters
3. **Intelligent Caching with Differentiated TTL** - Different data types, different refresh rates
4. **Market Hours Awareness** - Preventing stale data issues
5. **Graceful Degradation** - Maintaining service availability during API failures
6. **API Cost Optimization** - Achieving 60%+ cache hit rates

### Prerequisites

- Python 3.11+
- Redis (optional - for caching)
- Basic understanding of financial markets
- Completion of Finance AI M7 series (helpful but not required)

In [ ]:
# OFFLINE Mode Guard
import os

REDIS_ENABLED = os.getenv('REDIS_ENABLED', 'false').lower() == 'true'
OPENAI_ENABLED = os.getenv('OPENAI_ENABLED', 'false').lower() == 'true'

print("Service Status:")
print(f"  Redis: {'✅ Enabled' if REDIS_ENABLED else '❌ Disabled (caching unavailable)'}")
print(f"  OpenAI: {'✅ Enabled' if OPENAI_ENABLED else '❌ Disabled (full RAG unavailable)'}")
print("")
print("Note: yfinance (market data) is always available - no API key needed!")
print("")

if not REDIS_ENABLED:
    print("⚠️ OFFLINE MODE: Redis caching disabled")
    print("   System will work but without cost optimizations")
    print("   To enable: Set REDIS_ENABLED=true and run Redis locally")

## Section 1: Setup & Imports

First, let's import all necessary libraries and set up our environment.

In [ ]:
# Core imports
import sys
sys.path.append('..')  # Add parent directory to path

from src.l3_m8_financial_domain_knowledge_injection import (
    FinancialDataEnricher,
    FinancialRAGWithEnrichment,
    extract_tickers,
    is_market_open
)

# Standard library
import json
from datetime import datetime

# Optional: Redis for caching
try:
    import redis
    redis_available = True
except ImportError:
    redis_available = False
    print("⚠️ Redis library not installed. Caching will be disabled.")
    print("   To install: pip install redis")

print("✅ Imports successful")

## Section 2: Understanding Data Freshness Hierarchy

Not all financial data has the same freshness requirements. Let's explore the hierarchy:

### Tier 1: High-Frequency Trading Data (1-second refresh)
- **Use case:** Algorithmic trading, day trading
- **Cost:** Bloomberg Terminal ($24K/year per seat)
- **When justified:** Split-second trading decisions

### Tier 2: Analyst/Advisory Data (1-5 minute refresh)
- **Use case:** Investment research, portfolio management (← THIS MODULE)
- **Cost:** Free APIs like yfinance (15-min delay)
- **When justified:** Most RAG use cases

### Tier 3: Fundamental Company Data (24-hour refresh)
- **Use case:** Long-term investment analysis
- **Cost:** SEC EDGAR API (free)
- **When justified:** Earnings reports, 10-K filings

In [ ]:
# Demonstrate freshness hierarchy with TTL configuration
ttl_hierarchy = {
    "stock_price": 60,        # 1 minute - Tier 2: Real-time trading needs
    "company_info": 86400,    # 24 hours - Tier 3: Fundamentals change slowly
    "market_status": 300      # 5 minutes - Market hours don't change often
}

print("TTL Configuration (Time To Live in cache):")
print("="*50)
for data_type, ttl in ttl_hierarchy.items():
    minutes = ttl / 60
    hours = ttl / 3600
    if hours >= 1:
        print(f"  {data_type:20s}: {ttl:6d} seconds ({hours:.1f} hours)")
    else:
        print(f"  {data_type:20s}: {ttl:6d} seconds ({minutes:.1f} minutes)")

print("\nWhy these specific values?")
print("  - Stock prices: 1 min balances freshness vs cache hit rate")
print("  - Company info: 24 hours because fundamentals rarely change daily")
print("  - Market status: 5 min is reasonable for open/closed detection")

## Section 3: Initialize the Enricher

Let's create our FinancialDataEnricher instance, with or without Redis caching.

In [ ]:
# Initialize Redis client (if available)
redis_client = None

if REDIS_ENABLED and redis_available:
    try:
        redis_client = redis.Redis(
            host=os.getenv('REDIS_HOST', 'localhost'),
            port=int(os.getenv('REDIS_PORT', 6379)),
            db=0
        )
        redis_client.ping()
        print("✅ Redis connection successful")
    except Exception as e:
        print(f"⚠️ Redis connection failed: {e}")
        print("   Continuing without caching...")
        redis_client = None
else:
    print("ℹ️ Redis disabled - running without cache")

# Create enricher
enricher = FinancialDataEnricher(redis_client)

print(f"\n✅ FinancialDataEnricher initialized")
print(f"   Caching: {'Enabled' if redis_client else 'Disabled'}")

## Section 4: Check Market Status

Before enriching data, let's check if the market is currently open.
This prevents showing stale after-hours data as "current".

In [ ]:
# Check if market is open
market_open = is_market_open()

print(f"Market Status: {'🟢 OPEN' if market_open else '🔴 CLOSED'}")
print(f"Current time (UTC): {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')}")

# Get detailed status
status = enricher._get_market_status()
print(f"Detailed status: {status}")

print("\nMarket Hours (US Stock Market - Eastern Time):")
print("  Pre-Market:  4:00 AM - 9:30 AM")
print("  Regular:     9:30 AM - 4:00 PM")
print("  After-Hours: 4:00 PM - 8:00 PM")
print("  Closed:      Weekends & Holidays")

## Section 5: Extract Tickers from Text

Before enriching, we need to identify which stock tickers are mentioned in the text.

In [ ]:
# Sample texts with ticker mentions
sample_texts = [
    "AAPL and MSFT are performing well in the tech sector",
    "Tesla (TSLA) announced new delivery numbers",
    "The portfolio includes GOOGL, AMZN, and NVDA",
    "I think the market will go up"  # No tickers
]

print("Ticker Extraction Examples:")
print("="*60)

for text in sample_texts:
    tickers = extract_tickers(text)
    print(f"\nText: {text}")
    print(f"Tickers: {tickers if tickers else 'None found'}")

# Expected output:
# Text 1: ['AAPL', 'MSFT']
# Text 2: ['TSLA']
# Text 3: ['GOOGL', 'AMZN', 'NVDA']
# Text 4: [] (common words like 'I' are filtered)

## Section 6: Enrich with Real-Time Market Data

Now let's enrich a document with real-time market data from yfinance.

**Note:** yfinance is free but has a 15-minute delay. For most research use cases, this is acceptable.

In [ ]:
# Sample document from RAG retrieval
document_text = """
Apple Inc. reported strong Q1 2024 earnings with revenue of $94.9B,
beating analyst expectations of $93.2B. The company saw growth across
all product segments, with iPhone sales driving the majority of revenue.
"""

# Extract tickers
tickers = extract_tickers(document_text)
print(f"Tickers found: {tickers}")

# If no tickers found, use manual list
if not tickers:
    tickers = ["AAPL"]  # We know this document is about Apple
    print(f"Using manual tickers: {tickers}")

print("\nEnriching document with market data...")
print("(This may take a few seconds on first call)\n")

# Enrich with market data
result = enricher.enrich_with_market_data(document_text, tickers)

# Display results
print("="*60)
print("ENRICHMENT RESULT")
print("="*60)

for ticker, data in result['enriched_data'].items():
    if data.get('fallback'):
        print(f"\n{ticker}: ⚠️ Data unavailable (fallback mode)")
        print(f"  Error: {data.get('error')}")
    else:
        print(f"\n{ticker}:")
        print(f"  Current Price:  ${data.get('current_price')}")
        print(f"  Change:         {data.get('change_percent'):+.2f}% (${data.get('change_dollars'):+.2f})")
        print(f"  Market Cap:     {data.get('market_cap')}")
        print(f"  P/E Ratio:      {data.get('pe_ratio'):.2f}" if data.get('pe_ratio') else "  P/E Ratio:      N/A")
        print(f"  Volume:         {data.get('volume'):,}" if data.get('volume') else "  Volume:         N/A")
        print(f"  52W High/Low:   ${data.get('52_week_high')} / ${data.get('52_week_low')}")
        print(f"  Market Status:  {data.get('market_status')}")
        print(f"  Data Source:    {data.get('data_source')}")
        print(f"  Timestamp:      {data.get('data_timestamp')}")

print(f"\nCache Hit Rate: {result['cache_hit_rate']:.1f}%")
print(f"Enrichment Timestamp: {result['enrichment_timestamp']}")

## Section 7: Demonstrate Caching Benefits

Let's see the performance difference between cache hits and cache misses.

In [ ]:
import time

test_ticker = "MSFT"

# First call - likely cache miss (API call)
start = time.time()
result1 = enricher.enrich_with_market_data("Test", [test_ticker])
time1 = (time.time() - start) * 1000  # Convert to milliseconds

# Second call within TTL window - should be cache hit
start = time.time()
result2 = enricher.enrich_with_market_data("Test", [test_ticker])
time2 = (time.time() - start) * 1000

print("Caching Performance Comparison:")
print("="*60)
print(f"First call (cache miss):  {time1:.1f}ms")
print(f"Second call (cache hit):  {time2:.1f}ms")
print(f"Speedup: {time1/time2:.1f}x faster")

if redis_client:
    print("\n✅ Caching is working!")
    print(f"   Expected: ~200ms (API) vs ~10ms (cache)")
else:
    print("\n⚠️ Caching disabled - both calls hit the API")
    print("   Enable Redis to see 10-20x speedup")

## Section 8: Multiple Tickers Enrichment

Real-world scenarios often involve multiple stocks. Let's enrich a portfolio.

In [ ]:
# Portfolio analysis scenario
portfolio_text = """
My portfolio consists of major tech stocks:
- Apple (AAPL): 100 shares
- Microsoft (MSFT): 50 shares
- Google (GOOGL): 25 shares
"""

portfolio_tickers = ["AAPL", "MSFT", "GOOGL"]

print("Enriching portfolio with current market data...\n")

result = enricher.enrich_with_market_data(portfolio_text, portfolio_tickers)

print("Portfolio Market Data:")
print("="*80)

total_value = 0
shares = {"AAPL": 100, "MSFT": 50, "GOOGL": 25}

for ticker, data in result['enriched_data'].items():
    if not data.get('fallback'):
        price = data.get('current_price', 0)
        holding_value = price * shares.get(ticker, 0)
        total_value += holding_value

        print(f"\n{ticker}:")
        print(f"  Price:     ${price:,.2f}")
        print(f"  Shares:    {shares[ticker]}")
        print(f"  Value:     ${holding_value:,.2f}")
        print(f"  Change:    {data.get('change_percent'):+.2f}%")

print(f"\n{'='*80}")
print(f"Total Portfolio Value: ${total_value:,.2f}")
print(f"\nCache Hit Rate: {result['cache_hit_rate']:.1f}%")

## Section 9: RAG Integration Example

Now let's see how this integrates with a RAG system.
The `FinancialRAGWithEnrichment` class automatically extracts tickers and enriches context.

In [ ]:
# Initialize RAG system with enrichment
rag_system = FinancialRAGWithEnrichment(redis_client)

# User query
user_query = "How is Apple performing today?"

# Simulated RAG retrieval (in production, this comes from vector DB)
context = """
Apple Inc. is a technology company that designs and manufactures consumer electronics,
software, and online services. The company recently reported Q1 2024 earnings.
"""

print(f"User Query: {user_query}")
print(f"\nRetrieved Context (from vector DB):")
print(context)

# Execute RAG query with auto-enrichment
print("\nEnriching context with real-time data...")
result = rag_system.query(user_query, context)

print("\n" + "="*80)
print("RAG QUERY RESULT")
print("="*80)

print(f"\nTickers Found: {result['tickers_found']}")

for ticker, data in result['enrichment']['enriched_data'].items():
    if not data.get('fallback'):
        print(f"\n{ticker} Current Data:")
        print(f"  Price: ${data.get('current_price')} ({data.get('change_percent'):+.2f}%)")
        print(f"  Market Status: {data.get('market_status')}")

print("\n✅ Context is now enriched and ready for LLM generation")
print("   In production, you would pass this to GPT-4 for final response")

## Section 10: Monitoring & Metrics

Production systems need monitoring. Let's check our enrichment metrics.

In [ ]:
# Get metrics
metrics = enricher.get_metrics()

print("Enrichment Service Metrics:")
print("="*60)
print(f"\nCache Performance:")
print(f"  Hit Rate:        {metrics['cache_hit_rate']:.1f}% (target: 60%+)")
print(f"  Cache Hits:      {metrics['cache_hits']}")
print(f"  Cache Misses:    {metrics['cache_misses']}")

print(f"\nAPI Performance:")
print(f"  Total Calls:     {metrics['total_api_calls']}")
print(f"  Failure Rate:    {metrics['api_failure_rate']:.1f}%")

# Performance assessment
print("\nAssessment:")
if metrics['cache_hit_rate'] >= 60:
    print("  ✅ Cache hit rate is excellent (60%+ target achieved)")
elif metrics['cache_hit_rate'] >= 40:
    print("  ⚠️ Cache hit rate is acceptable (40-60% range)")
else:
    print("  ❌ Cache hit rate is low (<40% - consider increasing TTLs)")

if metrics['api_failure_rate'] <= 5:
    print("  ✅ API failure rate is healthy (<5%)")
else:
    print(f"  ⚠️ API failure rate is high ({metrics['api_failure_rate']:.1f}% - investigate)")

## Section 11: Error Handling & Fallback Behavior

Let's test how the system handles failures gracefully.

In [ ]:
# Test with an invalid ticker
invalid_ticker = "INVALID123"

print(f"Testing fallback behavior with invalid ticker: {invalid_ticker}\n")

result = enricher.enrich_with_market_data("Test", [invalid_ticker])

ticker_data = result['enriched_data'][invalid_ticker]

print("Result:")
print(f"  Fallback Mode: {ticker_data.get('fallback')}")
print(f"  Error Message: {ticker_data.get('error')}")
print(f"  Current Price: {ticker_data.get('current_price')}")

print("\n✅ System degraded gracefully - did not crash!")
print("   This maintains RAG system availability even when market data fails")

## Section 12: Cost Analysis

Let's calculate the cost savings from caching.

In [ ]:
# Scenario: Medium Hedge Fund
# - 100 analysts
# - 15 queries/day per analyst
# - 4 tickers per query on average

analysts = 100
queries_per_day = 15
tickers_per_query = 4
cache_hit_rate = 0.65  # 65% (our target)

# Calculate daily API calls
total_queries = analysts * queries_per_day
total_ticker_lookups = total_queries * tickers_per_query
cache_misses = total_ticker_lookups * (1 - cache_hit_rate)

print("Cost Analysis: Medium Hedge Fund")
print("="*60)
print(f"\nUsage:")
print(f"  Analysts:            {analysts}")
print(f"  Queries/day:         {total_queries:,}")
print(f"  Ticker lookups/day:  {total_ticker_lookups:,}")

print(f"\nWith {cache_hit_rate*100:.0f}% Cache Hit Rate:")
print(f"  Cache hits/day:      {int(total_ticker_lookups * cache_hit_rate):,} (served from Redis)")
print(f"  API calls/day:       {int(cache_misses):,} (hit external API)")
print(f"  API calls/month:     {int(cache_misses * 30):,}")

print(f"\nWithout Caching:")
print(f"  API calls/day:       {total_ticker_lookups:,}")
print(f"  API calls/month:     {total_ticker_lookups * 30:,}")

savings_percent = cache_hit_rate * 100
print(f"\n💰 Cost Savings: {savings_percent:.0f}% reduction in API calls")
print(f"   Equivalent to saving ${savings_percent*10:.0f}/month in API costs")

print("\nInfrastructure Costs:")
print("  Redis:       ₹2,000/month (AWS ElastiCache, 5GB)")
print("  Alpha V:     ₹4,200/month (Premium tier)")
print("  Total:       ₹6,200/month (~$75 USD)")
print("\n  vs. Bloomberg: ₹20,00,000/month for 100 seats ($24K/seat/year)")

## Summary & Key Takeaways

### What We Built

✅ **Real-time financial data enrichment system** that:
- Integrates yfinance API for live market data (free, 15-min delay)
- Implements intelligent caching with differentiated TTLs
- Handles market hours to prevent stale data issues
- Degrades gracefully when APIs fail
- Achieves 60%+ cache hit rate for cost optimization

### Key Concepts Learned

1. **Data Freshness Hierarchy** - Different data types need different refresh rates
2. **Cache-First Strategy** - Always check Redis (10ms) before APIs (200ms)
3. **Differentiated TTL** - Stock prices (1 min), company info (24 hours)
4. **Market Hours Awareness** - Don't show Friday 4 PM as Monday 10 AM current price
5. **Graceful Degradation** - API failures don't crash the RAG system

### Production Metrics

- **Target cache hit rate:** 60%+ (achieves 60% cost reduction)
- **Target latency:** P95 < 100ms (cached), P95 < 300ms (uncached)
- **API rate limits:** yfinance ~2,000 calls/hour
- **Cost savings:** 60% reduction vs. no caching

### When to Use This Approach

✅ **Use when:**
- Building investment research/advisory RAG systems
- Users need "current" prices (5-15 min delay acceptable)
- Queries mention: "current", "today", "now", "performing"
- Portfolio tracking and performance analysis

❌ **Don't use when:**
- Historical analysis only ("How did Apple perform in 2020?")
- Static company research ("Who is the CEO?")
- Regulatory document analysis (10-K, 10-Q parsing)
- High-frequency trading (need Bloomberg, not 15-min delay)

### Next Steps

1. **Deploy to production** with Redis for caching
2. **Monitor metrics** - cache hit rate, API failure rate, latency
3. **Tune TTLs** based on your specific use case
4. **Consider paid APIs** when free tier rate limits become problematic
5. **Integrate with vector DB** (Pinecone) for full RAG functionality

### Resources

- **Script:** [Augmented_Finance_AI_M8_2_RealTime_Financial_Data_Enrichment (1).md](https://github.com/yesvisare/financial_ai_ccc_l2/blob/main/Augmented_Finance_AI_M8_2_RealTime_Financial_Data_Enrichment%20(1).md)
- **yfinance Docs:** https://pypi.org/project/yfinance/
- **Redis Docs:** https://redis.io/docs/
- **FastAPI Docs:** https://fastapi.tiangolo.com/

---

**Congratulations!** You've built a production-ready financial data enrichment system. 🎉